In [3]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    TODO write the description
'''

datadir = "/datasets/eurora_data/db1/"
infile_jobs_to_nodes = datadir + "job_nodes.csv"
# infile_jobs = datadir + "april_long_jobs.csv"

infile_nodes = datadir + "nodes.csv"

# settings
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
suffix = "_1min_"

infile_jobs = datadir + "april_short_jobs.csv"

Along the nodes we have three differentIntel Xeon E5 processors stepping: nodes 1-16 & 25-32 havea maximum frequency of 2.1GHz (CPUs-2100), nodes 17-24 have a maximum frequency of 2.2GHz (CPUs-2200), andnodes 33-64 have a maximum frequency of 3.1GHz (CPUs-3100).

In [4]:
nodes_data = pd.read_csv(infile_nodes, index_col=0)
# nodes_data

### Read job2nodes data

In [5]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes, index_col=0)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data contains 469095 records


#### solve the problem of jobs runned on node 129

In [6]:
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129]
print("jobs_to_nodes_whole_data now contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data now contains 469020 records


#### solve the problem of duplicates job_id_string, node_id
this is probably caused by the concurrency writing of the database 

In [7]:
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("jobs_to_nodes_whole_data now contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data now contains 444610 records


### Read jobs data

In [8]:
short_jobs = pd.read_csv(infile_jobs, index_col=0)
print("jobs_whole_data contains {} records".format(short_jobs.shape[0]))

jobs_whole_data contains 66179 records


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
# merged_jobs_to_nodes[merged_jobs_to_nodes['job_id_string'].isin(short_jobs['job_id_string'])].shape

#### merge

In [50]:
merged_jobs_to_nodes = pd.merge(jobs_to_nodes_whole_data, short_jobs, on='job_id_string')
print(merged_jobs_to_nodes.shape[0])

68049


In [51]:
merged_jobs_to_nodes

,node_id,job_id_string,ncpus,ngpus,nmics,mem_requested,job_id,job_name,queue,start_time,...,node_req,cpu_req,mem_req,time_req,deleted,dependency,exit_status,pbs_exit_code,gpu_req,mic_req
0,44,498512.node129,12,0,0,14680064,47,job24.sh,parallel,2014-03-31 09:08:50,...,1,12,14680064,04:00,NaN,NaN,COMPLETED,NaN,0.0,0.0
1,9,498529.node129,16,0,0,10485760,60,ISQ-S1-PERP,parallel,2014-03-31 09:35:55,...,1,16,10,04:00,NaN,NaN,COMPLETED,NaN,NaN,NaN
2,16,498546.node129,16,0,0,5242880,71,life_perf_test,parallel,2014-03-31 10:25:32,...,8,128,40,01:00,NaN,NaN,COMPLETED,NaN,NaN,NaN
3,15,498546.node129,16,0,0,5242880,71,life_perf_test,parallel,2014-03-31 10:25:32,...,8,128,40,01:00,NaN,NaN,COMPLETED,NaN,NaN,NaN
4,8,498546.node129,16,0,0,5242880,71,life_perf_test,parallel,2014-03-31 10:25:32,...,8,128,40,01:00,NaN,NaN,COMPLETED,NaN,NaN,NaN
5,20,498546.node129,16,0,0,5242880,71,life_perf_test,parallel,2014-03-31 10:25:32,...,8,128,40,01:00,NaN,NaN,COMPLETED,NaN,NaN,NaN
6,24,498546.node129,16,0,0,5242880,71,life_perf_test,parallel,2014-03-31 10:25:32,...,8,128,40,01:00,NaN,NaN,COMPLETED,NaN,NaN,NaN
7,21,498546.node129,16,0,0,5242880,71,life_perf_test,parallel,2014-03-31 10:25:32,...,8,128,40,01:00,NaN,NaN,COMPLETED,NaN,NaN,NaN
8,32,498546.node129,16,0,0,5242880,71,life_perf_test,parallel,2014-03-31 10:25:32,...,8,128,40,01:00,NaN,NaN,COMPLETED,NaN,NaN,NaN
9,28,498546.node129,16,0,0,5242880,71,life_perf_test,parallel,2014-03-31 10:25:32,...,8,128,40,01:00,NaN,NaN,COMPLETED,NaN,NaN,NaN


In [52]:
#df = merged_jobs_to_nodes.groupby(['job_id_string']).size().reset_index(name='counts')
#df = df.sort_values('counts')
#df = df[df['counts'] > 1]
#print("{} jobs runned on more than one nodes".format(df.shape[0]))
#df

### compute the real power consumption

In [53]:
### remove to continue from a specific job

#long_jobs = long_jobs.drop(['real_pow', 'runned_alone', 'real_pow_quality'], axis=1)

short_jobs['real_pow'] = 0.0
short_jobs['runned_alone'] = True
short_jobs['real_pow_quality'] = 0.0

short_jobs['n_2_1'] = 0
short_jobs['n_2_2'] = 0
short_jobs['n_3_1'] = 0

In [54]:
#large_jobs.index[large_jobs.shape[0]-1]
short_jobs.head(1)

,job_id,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,...,exit_status,pbs_exit_code,gpu_req,mic_req,real_pow,runned_alone,real_pow_quality,n_2_1,n_2_2,n_3_1
201411,47,498512.node129,job24.sh,parallel,2014-03-31 09:08:50,2014-04-02 02:56:19,2014-04-02 02:56:22,emacerat@node129.eurora.cineca.it,1,12,...,COMPLETED,NaN,0.0,0.0,0.0,True,0.0,0,0,0


In [55]:
#print(real_pow.shape[0])
#print(runned_alone.shape[0])
#print(quality.shape[0])

#large_jobs_tail = large_jobs[large_jobs.index > 390345]

In [56]:
#large_jobs_tail.shape

In [2]:
i = 0
outfile = datadir + 'CPUs/' + interval_comment + "/short_jobs_real_pow.csv"
for index, row in short_jobs.iterrows():
#for index, row in large_jobs_tail.iterrows():
    print()
    print("{}/{}".format(i, short_jobs.shape[0]))

    #real_pow[i], runned_alone[i], quality[i] = calculate_job_consumption(row['job_id_string'])
    real_pow_val, runned_alone_val, quality_val, job_n_2_1, job_n_2_2, job_n_3_1 = calculate_job_consumption_2(row['job_id_string'])
    short_jobs.at[index,'real_pow'] = real_pow_val
    short_jobs.at[index,'runned_alone'] = runned_alone_val
    short_jobs.at[index,'real_pow_quality'] = quality_val
    
    short_jobs.at[index,'n_2_1'] = job_n_2_1
    short_jobs.at[index,'n_2_2'] = job_n_2_2
    short_jobs.at[index,'n_3_1'] = job_n_3_1
    
    i = i + 1
    if(i % 1000 == 0):
        short_jobs.to_csv(outfile)
        
    #    break

#large_jobs['real_pow'] = real_pow
#large_jobs['runned_alone'] = runned_alone
#large_jobs['pow_quality'] = quality
# write
short_jobs.to_csv(outfile)



NameError: name 'datadir' is not defined

In [44]:
#large_jobs.to_csv(outfile)

In [49]:
#large_jobs['real_pow'] = real_pow
#large_jobs['runned_alone'] = runned_alone
#large_jobs['pow_quality'] = quality
# write
#outfile = datadir + 'CPUs/' + interval_comment + "/large_jobs_real_pow_3.csv"
#large_jobs.to_csv(outfile)

In [59]:
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "01" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_01 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "02" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_02 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "03" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_03 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "04" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_04 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "05" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_05 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "06" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_06 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "07" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_07 = pd.read_csv(infile_node)

infile_node = datadir + "CPUs/" + interval_comment + "/node" + "08" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_08 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "09" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_09 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "10" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_10 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "11" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_11 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "12" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_12 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "13" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_13 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "14" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_14 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "15" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_15= pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "16" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_16 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "17" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_17 = pd.read_csv(infile_node)

In [60]:
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "18" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_18 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "19" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_19 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "20" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_20 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "21" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_21 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "22" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_22 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "23" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_23 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "24" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_24 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "25" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_25 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "26" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_26 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "27" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_27 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "28" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_28 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "29" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_29 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "30" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_30 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "31" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_31 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "32" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_32 = pd.read_csv(infile_node)

In [61]:
# most used nodes

infile_node = datadir + "CPUs/" + interval_comment + "/node" + "33" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_33 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "34" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_34 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "35" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_35 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "36" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_36 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "37" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_37 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "38" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_38 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "39" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_39 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "40" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_40 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "41" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_41 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "42" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_42 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "44" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_44 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "45" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_45 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "46" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_46 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "47" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_47 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "48" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_48 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "49" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_49 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "50" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_50 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "51" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_51 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "52" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_52 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "53" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_53 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "54" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_54 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "55" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_55 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "56" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_56 = pd.read_csv(infile_node)

In [62]:
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "57" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_57 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "58" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_58 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "59" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_59 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "60" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_60 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "61" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_61 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "62" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_62 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "63" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_63 = pd.read_csv(infile_node)
infile_node = datadir + "CPUs/" + interval_comment + "/node" + "64" + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements_64 = pd.read_csv(infile_node)



In [1]:
### TODO I probably have to load more than one node measurements file
def calculate_job_consumption_2(job_id_string):
    
    job_real_pow_consumption = 0
    job_runned_alone = True
    job_pow_quality = 1.0
    job_good_nodes = 0 # node where we have at most 1 measurement in the interval
    job_measurements_tot = 0
    job_measurements_missing_tot = 0
    
    partial_pow_consumption = 0
    
    job_to_nodes = merged_jobs_to_nodes[merged_jobs_to_nodes['job_id_string'] == job_id_string]
    job_row = job_to_nodes.iloc[0]
    job_to_nodes_unique = job_to_nodes.node_id.unique() # series with the nodes id
    ### print some jobs info
    print("job {} use {} cores in this node".format(job_id_string, str(job_row['ncpus'])))
    #print("job {} start at {} and end at {}".format(job_id_string, str(job_row['run_start_time']), str(job_row['end_time'])))
    print("job {} requires {} nodes and {} cores".format(job_id_string, str(job_row['node_req']), str(job_row['cpu_req'])))
    print(job_to_nodes_unique)
    
    job_n_2_1 = 0
    job_n_2_2 = 0
    job_n_3_1 = 0
    
    for index, row in job_to_nodes.iterrows():
        
        node_type = nodes_data.iloc[row['node_id']-1]['cpu_type']
        if(node_type == '2_1_ghz'):
            job_n_2_1 += 1
        elif(node_type == '2_2_ghz'):
            job_n_2_2 += 1
        elif(node_type == '3_1_ghz'):
            job_n_3_1 += 1
        
        node_idle_core = nodes_data.iloc[row['node_id']-1]['core_idle']
        
        if(row['node_id'] == 1):
            node_measurements = node_measurements_01
        elif(row['node_id'] == 2):
            node_measurements = node_measurements_02
        elif(row['node_id'] == 3):
            node_measurements = node_measurements_03
        elif(row['node_id'] == 4):
            node_measurements = node_measurements_04
        elif(row['node_id'] == 5):
            node_measurements = node_measurements_05
        elif(row['node_id'] == 6):
            node_measurements = node_measurements_06
        elif(row['node_id'] == 7):
            node_measurements = node_measurements_07
        elif(row['node_id'] == 8):
            node_measurements = node_measurements_08
        elif(row['node_id'] == 9):
            node_measurements = node_measurements_09
        elif(row['node_id'] == 10):
            node_measurements = node_measurements_10
        elif(row['node_id'] == 11):
            node_measurements = node_measurements_11
        elif(row['node_id'] == 12):
            node_measurements = node_measurements_12
        elif(row['node_id'] == 13):
            node_measurements = node_measurements_13
        elif(row['node_id'] == 14):
            node_measurements = node_measurements_14
        elif(row['node_id'] == 15):
            node_measurements = node_measurements_15
        elif(row['node_id'] == 16):
            node_measurements = node_measurements_16
        elif(row['node_id'] == 17):
            node_measurements = node_measurements_17
        elif(row['node_id'] == 18):
            node_measurements = node_measurements_18
        elif(row['node_id'] == 19):
            node_measurements = node_measurements_19
        elif(row['node_id'] == 20):
            node_measurements = node_measurements_20
        elif(row['node_id'] == 21):
            node_measurements = node_measurements_21
        elif(row['node_id'] == 22):
            node_measurements = node_measurements_22
        elif(row['node_id'] == 23):
            node_measurements = node_measurements_23
        elif(row['node_id'] == 24):
            node_measurements = node_measurements_24
        elif(row['node_id'] == 25):
            node_measurements = node_measurements_25
        elif(row['node_id'] == 26):
            node_measurements = node_measurements_26
        elif(row['node_id'] == 27):
            node_measurements = node_measurements_27
        elif(row['node_id'] == 28):
            node_measurements = node_measurements_28
        elif(row['node_id'] == 29):
            node_measurements = node_measurements_29
        elif(row['node_id'] == 30):
            node_measurements = node_measurements_30
        elif(row['node_id'] == 31):
            node_measurements = node_measurements_31
        elif(row['node_id'] == 32):
            node_measurements = node_measurements_32
        elif(row['node_id'] == 33):
            node_measurements = node_measurements_33
        elif(row['node_id'] == 34):
            node_measurements = node_measurements_34
        elif(row['node_id'] == 35):
            node_measurements = node_measurements_35
        elif(row['node_id'] == 36):
            node_measurements = node_measurements_36
        elif(row['node_id'] == 37):
            node_measurements = node_measurements_37
        elif(row['node_id'] == 38):
            node_measurements = node_measurements_38
        elif(row['node_id'] == 39):
            node_measurements = node_measurements_39
        elif(row['node_id'] == 40):
            node_measurements = node_measurements_40
        elif(row['node_id'] == 41):
            node_measurements = node_measurements_41
        elif(row['node_id'] == 42):
            node_measurements = node_measurements_42
        elif(row['node_id'] == 44):
            node_measurements = node_measurements_44
        elif(row['node_id'] == 45):
            node_measurements = node_measurements_45
        elif(row['node_id'] == 46):
            node_measurements = node_measurements_46
        elif(row['node_id'] == 47):
            node_measurements = node_measurements_47
        elif(row['node_id'] == 48):
            node_measurements = node_measurements_48
        elif(row['node_id'] == 49):
            node_measurements = node_measurements_49
        elif(row['node_id'] == 50):
            node_measurements = node_measurements_50
        elif(row['node_id'] == 51):
            node_measurements = node_measurements_51
        elif(row['node_id'] == 52):
            node_measurements = node_measurements_52
        elif(row['node_id'] == 53):
            node_measurements = node_measurements_53
        elif(row['node_id'] == 54):
            node_measurements = node_measurements_54
        elif(row['node_id'] == 55):
            node_measurements = node_measurements_55
        elif(row['node_id'] == 56):
            node_measurements = node_measurements_56
        elif(row['node_id'] == 57):
            node_measurements = node_measurements_57
        elif(row['node_id'] == 58):
            node_measurements = node_measurements_58
        elif(row['node_id'] == 59):
            node_measurements = node_measurements_59
        elif(row['node_id'] == 60):
            node_measurements = node_measurements_60
        elif(row['node_id'] == 61):
            node_measurements = node_measurements_61
        elif(row['node_id'] == 62):
            node_measurements = node_measurements_62
        elif(row['node_id'] == 63):
            node_measurements = node_measurements_63
        elif(row['node_id'] == 64):
            node_measurements = node_measurements_64
        
        
        else:
            ### add a 0 to the number if is less than 10 ("1" -> "01", ...)
            node_str = "0" + str(row['node_id']) if row['node_id'] < 10 else str(row['node_id'])
            print("node {}".format(node_str))
            ### open the measurements node file I/O
            infile_node = datadir + "CPUs/" + interval_comment + "/node" + node_str + suffix + interval_comment + "_active_cores_and_jobs.csv"
            node_measurements = pd.read_csv(infile_node)
    
        measure = node_measurements[node_measurements['timestamp'] == job_row['run_start_time'][:-3]+':00']
        
        if(measure.shape[0] != 0):
            if(measure['active_cores'].iloc[0] != 0):
                job_runned_alone = False
            
            active_cores = measure['active_cores'].iloc[0] + row['ncpus']
            if(active_cores > 16):
                active_cores = 16
            job_real_pow_consumption = job_real_pow_consumption + (measure['pow_tot_0'].iloc[0] + measure['pow_tot_1'].iloc[0] - (16 - active_cores) * node_idle_core) * row['ncpus'] / active_cores
        else:
            print("error measure")
            job_pow_quality = 0.0
    
    if(job_real_pow_consumption < 0):
        job_real_pow_consumption = 0
        job_pow_quality = 0.0
        
    return job_real_pow_consumption, job_runned_alone, job_pow_quality, job_n_2_1, job_n_2_2, job_n_3_1

In [30]:
node_str = "01"
print("node {}".format(node_str))
### open the measurements node file I/O
infile_node = datadir + "CPUs/" + interval_comment + "/node" + node_str + suffix + interval_comment + "_active_cores_and_jobs.csv"
node_measurements = pd.read_csv(infile_node)

node 01


In [31]:
node_measurements.head()

,Unnamed: 0,timestamp,pow_tot_0,pow_tot_1,active_cores,active_jobs,active_gpus,active_mics
0,0,2014-03-31 00:00:00,22.750122,24.923517,0,0,0,0
1,1,2014-03-31 00:01:00,22.493928,24.574456,0,0,0,0
2,2,2014-03-31 00:02:00,22.324329,24.492541,0,0,0,0
3,3,2014-03-31 00:03:00,21.853533,23.842877,0,0,0,0
4,4,2014-03-31 00:04:00,22.043767,23.952392,0,0,0,0


In [32]:
'2014-04-02 00:30:00'[:-3]+':00'

'2014-04-02 00:30:00'

In [36]:
mea = node_measurements[node_measurements['timestamp'] == '2014-04-02 00:30:00'[:-3]+':00']
mea

,Unnamed: 0,timestamp,pow_tot_0,pow_tot_1,active_cores,active_jobs,active_gpus,active_mics
2910,2910,2014-04-02 00:30:00,25.559256,25.524705,16,1,0,0


In [37]:
mea.iloc[0]['active_cores']

16

In [57]:
#### calculate the power consumption of a specifified job



job_id_string = "498462.node129" # non ha misurazioni
job_id_string = "498463.node129" # non ha misurazioni
job_id_string = "498464.node129" # non ha misurazioni
job_id_string = "498465.node129" # [ 35]

job_id_string = "498699.node129" # [ 37]
job_id_string = "498458.node129" # non ha misurazioni
job_id_string = "498458.node129" # non ha misurazioni
job_id_string = "498461.node129" # single node 16 cores [ 27]
job_id_string = "500752.node129" # 2 nodi, 24 cores [ 16 20] 20 totalmente assente
job_id_string = "500899.node129" # [ 15 23 32 28 25 19]

job_id_string = "498467.node129"

job_id_string = "498460.node129" # 6 nodi, 96 cores [ 9 28 25 19 26 29]
job_id_string = "498466.node129" # [ 36 38 40 42 50 56 59 61 62 63 54]
job_id_string = "500752.node129" # 2 nodi, 24 cores [ 16 20] 20 totalmente assente
job_id_string = "498459.node129" # single node 16 cores [ 9]
job_id_string = "498699.node129" # [ 37]
job_id_string = "498458.node129" # [ ]
job_id_string = "498460.node129" # 6 nodi, 96 cores [ 9 28 25 19 26 29]
job_id_string = "500899.node129" # [ 15 23 32 28 25 19]


print("job_id_string: {}".format(job_id_string))



job_id_string: 500899.node129


In [63]:
calculate_job_consumption_2(job_id_string)

job 500899.node129 use 16 cores in this node
job 500899.node129 start at 2014-03-31 19:24:06 and end at 2014-03-31 19:40:33
job 500899.node129 requires 6 nodes and 96 cores
[15 23 32 28 25 19]
node 15
0/15 intervals missing
   active_cores  active_jobs  active_gpus  active_mics  counts  pow_tot_0_mean  pow_tot_1_mean
0            16            1            0            0      15       94.476258         96.9878
partial measurement: 191.46405847759212
0/15 intervals missing
   active_cores  active_jobs  active_gpus  active_mics  counts  pow_tot_0_mean  pow_tot_1_mean
0            16            1            0            0      15       93.407682       92.983533
partial measurement: 186.3912153826789
node 32
15/15 intervals missing
partial measurement: 0
0/15 intervals missing
   active_cores  active_jobs  active_gpus  active_mics  counts  pow_tot_0_mean  pow_tot_1_mean
0            16            1            0            0      15       97.068548       98.201442
partial measurement: 195.2

(1147.5136811412594, True, 0.8333333333333334, 4, 2, 0)